# SUSI sweRun
This script runs Susi* for one or more than one sites in Sweden.
It tryes to facilitate a form to get the parameters, collects the smhi weather data and storage the outputs in a centralized folder per site.

Follow the link to susi ....
get the general instructions from ...

Alejandro G.
 /// I have to improve the description ///

In [9]:
# Importing libraries
# Local
import scripts.param_import  as ipft # script to import from xls to dict

# General
import numpy as np
import pandas as pd
from datetime import datetime
import configparser
import argparse
import matplotlib.pylab as plt
import seaborn as sns
import sys
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.misc import derivative
from scipy.interpolate import InterpolatedUnivariateSpline as interS
from scipy.sparse import  diags
from scipy.sparse.linalg import  spsolve
#----------------------------------------------------------------------------
# Install non-standard packages
# !pip install xlsxwriter
# !pip install xlrd==1.2.0
# !pip install pyproj
# #--------------------------
# !pip install netCDF4
# !pip install datetime
# !pip install xlrd --upgrade
# !pip install dataframe_image
# # --------------------------
from pyproj import CRS, Transformer


# Imports susi
newpathsusi = 'susi_2022'
if newpathsusi not in sys.path: sys.path.insert(0,newpathsusi)

#swepath = 'inputs/sweden/param/'
#if swepath not in sys.path: sys.path.insert(0,swepath)
# -----------------------------------------------------------------------------
from susi.susi_utils import read_FMI_weather
from susi.susi_main import Susi
from inputs.susi_para import get_susi_para

#from inputs.susi_para_swe import get_susi_para
#import scripts.import_param_from_table as ipft

#exec(f"from {data_files[site]['para']} import get_susi_para")


print (f"SUSI is ready to run!")

SUSI is ready to run!


### Environment variables

In [4]:
# Git clone and environment var
# gets the parameter xls file name ("./parameters"), default "susi-param.xlsx"
# see structure in README.md

# Folders and names

# Name convention
  # parameter file:   inputs/sweden/param/    susi_para_{site}.py
  # weather file:     inputs/sweden/weather/  {site}_weather.csv
  # motti file:       inputs/sweden/motti/    {site}_motti_lyr_0.xlsx

param_file = "inputs\sweden\parameters.xlsx"

sNames = ["01_Bjdamm", "03_Korpis", "04_Fröjered", "05_SträRed", "06_Risabo", "07_Rothult",     # 0 - 5
                "08_Ydreham", "09_Gröngöl", "11_Ökalix", "12_Ökalix", "13_Ökalix",              # 6 - 10
                "14_Höglund", "15_Jörnmark"]                                                    # 11 - 12


# data_files ditrionary storages the 
    # motti refference for each site, please follow the names convention.
data_files = {}

for s in sNames:
    key = 'swe_site_'+s
    data_files[key] = {}
    data_files[key]['para'] = f'susi_para_{s}'
    data_files[key]['wFile'] = f'{s}_weather.csv'
    data_files[key]['motti_lyr_0'] = f'{s}_motti_input_lyr_0.xlsx'
    data_files[key]['motti_lyr_1'] = f'{s}_motti_input_lyr_0.xlsx'        # fake, just to fill the gap
    data_files[key]['motti_lyr_2'] = f'{s}_motti_input_lyr_0.xlsx'        # fake, just to fill the gap

# Repositories 
if not os.path.exists('susi_2022'): 
    !git clone https://github.com/annamarilauren/susi_2022
else: print("Susi repository is already in yor computer")

if not os.path.exists('susi_SMHI'): 
    !git clone https://github.com/alejandrogandara/susi_SMHI
else: print("SMHI collect repository is already in yor computer")



Susi repository is already in yor computer
SMHI collect repository is already in yor computer


### Default inputs

In [10]:
folderName='outputs/'                                        # output folder
inpath = 'inputs/sweden/weather/'
mottipath = 'inputs/sweden/motti/'                                            # folder where weather files are
mottifile = {'path':mottipath,
              'dominant':{1: data_files[site]['motti_lyr_0']},
              'subdominant':{0:data_files[site]['motti_lyr_2']},
              'under':{0:data_files[site]['motti_lyr_2']}}

graph_folder = 'outputs/graphs/'
wdata = data_files[site]['wFile']
# -------------------------------- UNTIL HERE
# imports variables from table

sarkaSim = 40.                                                                  # strip width, ie distance between ditches, m
start_date = datetime(2011,1,1)
end_date = datetime(2021,12,31)
#sarkaSim = start_date = end_date = None

vparam = ipft.getVariablesFromFile(param_file, site)  # AG, gets the parameter from the frist worksheet in xlsx

#sarkaSim = vparam['sarkaSim']
#start_date = vparam['start_date']
#end_date = vparam['end_date']

start_yr = start_date.year 
end_yr = end_date.year
yrs = (end_date - start_date).days/365.25
n = int(sarkaSim / 2)                                                           # number of computation nodes in the strip

ageSim = {'dominant': 45.*np.ones(n),
          'subdominant': 0*np.ones(n),
          'under': 0*np.ones(n)}                                                         # age of the stand in each node

sfc =  np.ones(n, dtype=int)*3                                                            # site fertility class


#ageSim['dominant'][int(n/2):] = 2.
#ageSim[4:-4] = 2.

          
wpara, cpara, org_para, spara, outpara, photopara = get_susi_para(wlocation='swe', peat='swe', 
                                                                          folderName=folderName, hdomSim=None,  
                                                                          ageSim=ageSim, sarkaSim=sarkaSim, sfc=sfc, susiPath = inpath,
                                                                          n=n)

forc=read_FMI_weather(0, start_date, end_date, sourcefile=inpath+wdata)           # read weather input

#spara['canopylayers']['dominant'][int(n/2):] = 2                                                                        
#spara['canopylayers']['subdominant'][:int(n/2)] = 1                                                                        


file does not exists


KeyError: 'swe'

### Which site are you running?

In [7]:
siteID = 3
site = list(data_files.keys())[siteID]
site

'swe_site_05_SträRed'